In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import os.path as osp
import scanpy as sc
from IPython.display import display
import sys
sys.path.insert(0, "..")
import matplotlib as mpl
from run import *


In [2]:
dataset_dir = "tm_droplet_Trachea"
highly_genes_num = 0.1
seed = 0
ggl_loss_weight = 1
gsl_L1_weight = 0.01
rec_loss_weight = 0.1
dropout_layer_prob = 0.2
output_dir = "imputation_output_ig"

In [3]:
lambda_c = float(lambda_c)
data_dir = "/mnt/e/czbiohub-tabula-muris"
fig_dir = osp.join('/mnt/e/imputation')
batch_size = 256
dim = 400
epochs = 500000
generate_files = False
gpu_option = "0"
ig = True
learning_rate = 1e-4
low_expression_threshold = 0.20
low_expression_percentage = 0.80
split_pct = "0.8"
compression = '.gz'
n_neighbors = 10
n_neg_sample = 20
chunk_size = 256
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
rng = default_rng(seed)
adata, adata_unscaled, adata_cnt, post_zero_mask = load_data(data_dir, dataset_dir, highly_genes_num, n_neighbors, rng, generate_files, seed, low_expression_threshold=low_expression_threshold, low_expression_percentage=low_expression_percentage)
if epochs > 0:
    valid_split = int(float(split_pct) * len(adata.X))
    if dim <= 1 and dim > 0:
        dim = int(adata.X.shape[1] * dim)
    elif dim > 1:
        dim = int(dim)
    else:
        raise
    dims = [adata.X.shape[1], dim]
    print(dataset_dir, adata.X.shape[0], adata.X.shape[1])
model = Model(data_dir, dataset_dir, output_dir, dims, learning_rate, batch_size, ggl_loss_weight, gsl_L1_weight, rec_loss_weight, dropout_layer_prob, epochs, seed, ig=ig)
num2type_df = pd.read_csv(osp.join(data_dir, dataset_dir, "0.1.num2typeandcount.name.csv.gz"), index_col=0)
selection_df = pd.read_csv(osp.join(data_dir, dataset_dir, output_dir, "gene_selection_weight.name.csv.gz"), index_col=0)
display(num2type_df)


tm_droplet_Trachea 11271 1679


,num2type,num2count,markers
0,blood cell,1139,Ptprc
1,endothelial cell,1028,Pecam1
2,epithelial cell,892,"Epcam, Cdh1"
3,mesenchymal cell,7848,"Pdgfrb, Pdgfra, Col1a1, Col8a1"
4,neuroendocrine cell,362,"Cck, Syp"


In [4]:
all_markers = []
type_marker_dict = {}
for idx in num2type_df.index:
# for type_markers in num2type_df['markers'].values:
    type_markers = num2type_df.loc[idx, 'markers']
    one_type_markers = [marker.rstrip('+').rstrip('-') for marker in type_markers.split(', ')]
    one_type_markers = [marker for marker in one_type_markers if marker not in all_markers]
    if dataset_dir == "tm_droplet_Trachea":
        not_selected_markers = ['Col1a1', 'Syp', 'Pdgfrb', 'Col8a1']
        one_type_markers = [marker for marker in one_type_markers if marker not in not_selected_markers] 
    type_marker_dict[num2type_df.loc[idx, 'num2type']] = one_type_markers
    all_markers.extend(one_type_markers)

In [5]:
type_marker_dict

{'blood cell': ['Ptprc'],
 'endothelial cell': ['Pecam1'],
 'epithelial cell': ['Epcam', 'Cdh1'],
 'mesenchymal cell': ['Pdgfra'],
 'neuroendocrine cell': ['Cck']}

In [ ]:
m_steps = 50
method = 'riemann_trapezoidal'
ig_batch_size = 64 # 32
target_genes = all_markers
target_gene_indices = [adata.var["gene_name"].tolist().index(gene) for gene in target_genes]
ig_post_zero_mask = None
ig_post_zero_mask = post_zero_mask
target_gene_ig_dict, baseline_imX = model.check_ig(adata, adata_unscaled, adata_cnt, gpu_option, target_genes, m_steps, method, ig_batch_size, post_zero_mask=ig_post_zero_mask)


In [11]:
top_gene_num = 5
for idx, (k, v) in enumerate(type_marker_dict.items()):
    for one_marker in v:
        ig_genes = adata_cnt[:, target_gene_ig_dict[one_marker].sum(0).argsort()[::-1][:top_gene_num]].var["gene_name"]
        print(k, one_marker)
        print(ig_genes)

blood cell Ptprc
490     Cytl1
1095      Mgp
504     Dcpp3
1273     Ppa1
503     Dcpp2
Name: gene_name, dtype: object
endothelial cell Pecam1
490      Cytl1
504      Dcpp3
503      Dcpp2
1095       Mgp
1068    Malat1
Name: gene_name, dtype: object
epithelial cell Epcam
1095      Mgp
503     Dcpp2
504     Dcpp3
502     Dcpp1
565       Eln
Name: gene_name, dtype: object
epithelial cell Cdh1
419       Col1a1
423       Col3a1
885       Igfbp4
1428    Serping1
1427    Serpinf1
Name: gene_name, dtype: object
mesenchymal cell Pdgfra
1095       Mgp
504      Dcpp3
503      Dcpp2
1068    Malat1
1049       Ltf
Name: gene_name, dtype: object
neuroendocrine cell Cck
490      Cytl1
333       Cd74
442      Crip1
1273      Ppa1
803     H2-Eb1
Name: gene_name, dtype: object
